# Data Preview

## 1. Set up

In [53]:
# Standard imports
from pathlib import Path
import os
import sys

def set_project_root():
    notebooks_dir = Path.cwd()

    # Calculate the root directory of the project (go up three levels)
    project_root = notebooks_dir.parent.parent.parent

    if str(project_root) not in sys.path:
        print(f"The root directory of the project is: {project_root}")
        sys.path.append(str(project_root))

    return project_root

project_root = set_project_root()

### 1.3 Importing Data

In [54]:
# Third-party imports
import numpy as np
import pandas as pd

# Local imports
from pipeline.src._csv_utils import DataPathCleaningManager
from pipeline.src._config import ConfigManager

config_file = ConfigManager("run_pipeline.conf")
TIMEPLACE = "MARKET_OFFERS_TIMEPLACE"
data_timeplace = config_file.read_value(TIMEPLACE)
if data_timeplace is None:
    raise ValueError(F"The environment variable {TIMEPLACE} is not set.")

data_path_manager = DataPathCleaningManager(data_timeplace, project_root)

df_otodom = data_path_manager.load_df(domain="otodom", is_cleaned=False)

### 1.2 Functions

In [55]:
def count_and_percentage(df, column_name):
    """
    Function to calculate the count and percentage of unique values in a given column of a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame to analyze.
    column_name (str): The name of the column in the DataFrame.

    Returns:
    pandas.DataFrame: A DataFrame with the count and percentage of each unique value in the specified column.

    Raises:
    ValueError: If the specified column is not found in the DataFrame.
    """
    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame.")

    # Calculate count and normalized values
    count = df[column_name].value_counts(dropna=False)
    normalized = df[column_name].value_counts(dropna=False, normalize=True) * 100

    # Concatenate count and normalized values side by side
    result = pd.concat([count, normalized], axis=1)
    result.columns = ['Count', 'Percentage']

    return result

In [56]:
def count_comma_separated_values(df, column_name):
    """
    Counts the occurrences of individual elements in a comma-separated string column of a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the column.
    column_name (str): The name of the column to analyze.

    Returns:
    pandas.DataFrame: A DataFrame with the count and percentage of each unique element found in the comma-separated values.

    Raises:
    ValueError: If the specified column is not found in the DataFrame.
    """
    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame.")

    # Split the column values, explode to individual elements, and count
    exploded_items = df[column_name].dropna().str.split(', ').explode()
    exploded_df = pd.DataFrame({column_name: exploded_items})
    counts_and_percent = count_and_percentage(exploded_df, column_name)

    return counts_and_percent

In [57]:
def remove_non_numeric_characters(df, column_name):
    """
    Removes all non-numeric characters from a column of a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the column.
    column_name (str): The name of the column to analyze.

    Returns:
    pandas.DataFrame: A DataFrame with all non-numeric characters removed from the specified column.

    Raises:
    ValueError: If the specified column is not found in the DataFrame.
    """

    return df[column_name].str.replace('[^a-zA-Z]', '', regex=True).unique()

In [58]:
def count_words(text):
    if pd.isna(text):
        return 0
    return len(str(text).split())


## 2. Data preview

### Otodom

#### 2.2.1 Cleaning data

In [59]:
def clean_otodom_data(df: pd.DataFrame):

    # 1. Split 'location' into street, city, and voivodeship
    df['location_split'] = df['location'].str.split(', ')
    df['street'] = df['location_split'].apply(lambda x: x[0] if len(x) > 2 else None)
    df['city'] = df['location_split'].apply(lambda x: x[-2] if len(x) > 1 else None)
    df['voivodeship'] = df['location_split'].apply(lambda x: x[-1] if x else None)

    # Drop the temporary 'location_split' column
    df.drop(columns=['location_split'], inplace=True)

    # 2. Convert 'price' into float
    df['price'] = df['price'].str.replace(' ', '').str.extract('(\d+)')[0]
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    df['price'] = df['price'].astype('float64')

    # Extract and convert 'square_meters' into integers
    df['square_meters'] = df['square_meters'].str.extract('(\d+)')[0].astype('float64')

    # Extract and convert 'rent' into float
    df['rent'] = df['rent'].str.extract('(\d+)')[0]
    df['rent'] = pd.to_numeric(df['rent'], errors='coerce').astype('float64')
    df['total_rent'] = df['rent'].add(df['price'], fill_value=0).astype('float64')

    # Extract and convert 'deposit' into float
    df['deposit'] = df['deposit'].str.replace(' ', '').str.extract('(\d+)')[0]
    df['deposit'] = pd.to_numeric(df['deposit'], errors='coerce').astype('float64')

    # Convert 'number_of_rooms' into an integer, special handling for "Kawalerka"
    df['number_of_rooms'] = df['number_of_rooms'].astype('Int64')

    # Extract and clean 'floor_level'
    df_split = df['floor_level'].str.split('/', expand=True)
    df_split[0] = df_split[0].replace({'parter': 0, 'suterena': -1, '> 10': 11})

    poddasze_rows = df_split[0] == 'poddasze'
    df_split.loc[poddasze_rows, 0] = (df_split.loc[poddasze_rows, 1].fillna(0).astype(int) + 1).astype(str)

    df['attic'] = df_split[0] == 'poddasze'
    df['floor'] = pd.to_numeric(df_split[0], errors='coerce')
    df['floor'] = df['floor'].astype('Int64')
    df['building_floors'] = pd.to_numeric(df_split[1], errors='coerce')
    df['building_floors'] = df['building_floors'].astype('Int64')
    
    del df['floor_level']

    # Convert 'elevator' and 'parking_space' into boolean values
    df['elevator'] = df['elevator'].map({'tak': True, 'nie': False}).astype('boolean')

    df['parking_space'] = df['parking_space'].map({'garaż/miejsce parkingowe': True, 'brak informacji': False}).astype('boolean')
    
    # Convert 'build_year' into integers
    df['build_year'] = pd.to_numeric(df['build_year'], errors='coerce').astype('Int64')

    # todo create master columns for subcolumns
    # 3. Explode 'equipment', 'media_types', 'heating', 'security', 'windows', 'building_materials', 'additional_information' into boolean categories
    def explode_and_get_dummies(column_name):
        return df[column_name].str.get_dummies(sep=', ')
    
    to_explode = ['equipment', 'media_types', 'heating', 'security', 'windows', 'balcony_garden_terrace', 'building_material', 'additional_information']

    for column in to_explode:
        df = df.join(explode_and_get_dummies(column).add_prefix(f"{column}_"))

    for column in to_explode:
        del df[column]

    return df


In [60]:
df_otodom_cleaned = clean_otodom_data(df_otodom)
df_otodom_cleaned.head()

,link,title,location,price,summary_description,square_meters,rent,number_of_rooms,deposit,building_type,...,security_monitoring / ochrona,security_teren zamknięty,windows_drewniane,windows_plastikowe,balcony_garden_terrace_balkon,building_material_cegła,building_material_żelbet,additional_information_brak informacji,additional_information_oddzielna kuchnia,additional_information_tylko dla niepalących
0,https://www.otodom.pl/pl/oferta/bezposrednio-w...,"BEZPOŚREDNIO, wola, 2 pokojowe","ul. Władysława Przanowskiego 83, Ulrychów, Wol...",3000.0,Wynajmę bezpośrednio mieszkanie w Warszawie pr...,36.0,NaN,2,NaN,blok,...,0,0,1,0,1,1,0,1,0,0
1,https://www.otodom.pl/pl/oferta/ciche-52m2przy...,Ciche 52m2przy kanale żerańskim - Bez Prowizj,"ul. Żeglugi Wiślanej, Kobiałka, Białołęka, War...",2340.0,Zamieszkaj w ciszy i dobrym standardzie\n\n ...,51.0,NaN,2,2940.0,blok,...,1,1,0,1,1,1,0,0,1,1
2,https://www.otodom.pl/pl/oferta/rezerwacja-m2-...,REZERWACJA - M2 44m2 | Balkon | Garaż | Praga ...,"ul. Ostrobramska, Gocław, Praga-Południe, Wars...",3200.0,REZERWACJA!!!\n\nOferuję do wynajęcia dwupokoj...,43.0,1.0,2,3200.0,blok,...,1,1,0,1,1,0,1,0,1,0
3,https://www.otodom.pl/pl/oferta/3-oddzielne-po...,3 oddzielne pok. - Park Picassa - Bez prowizji,"ul. Erazma z Zakroczymia, Tarchomin, Białołęka...",3000.0,Zamieszkaj w ciszy w dobrym standardzie na pon...,63.0,NaN,3,3550.0,blok,...,0,0,0,1,1,1,0,0,1,1


In [61]:
df_otodom_cleaned.columns.to_list()

['link',
 'title',
 'location',
 'price',
 'summary_description',
 'square_meters',
 'rent',
 'number_of_rooms',
 'deposit',
 'building_type',
 'available_from',
 'remote service',
 'completion',
 'ownership',
 'rent_to_students',
 'elevator',
 'parking_space',
 'build_year',
 'street',
 'city',
 'voivodeship',
 'total_rent',
 'attic',
 'floor',
 'building_floors',
 'equipment_kuchenka',
 'equipment_lodówka',
 'equipment_meble',
 'equipment_piekarnik',
 'equipment_pralka',
 'equipment_zmywarka',
 'media_types_brak informacji',
 'media_types_internet',
 'media_types_telewizja kablowa',
 'heating_miejskie',
 'security_brak informacji',
 'security_domofon / wideofon',
 'security_monitoring / ochrona',
 'security_teren zamknięty',
 'windows_drewniane',
 'windows_plastikowe',
 'balcony_garden_terrace_balkon',
 'building_material_cegła',
 'building_material_żelbet',
 'additional_information_brak informacji',
 'additional_information_oddzielna kuchnia',
 'additional_information_tylko dla niep

In [62]:
columns_order = [
    'link', 'title', 'summary_description', 'remote service', 
    'price', 'rent', 'total_rent', 'deposit', 
    'location', 'street', 'city', 'voivodeship', 
    'square_meters', 'number_of_rooms', 'floor', 'attic', 'building_floors', 
    'available_from', 'completion', 'ownership', 'rent_to_students', 
    'building_type', 'build_year', 
    'elevator', 'parking_space', 
    'equipment_brak informacji', 'equipment_kuchenka', 'equipment_lodówka', 'equipment_meble', 'equipment_piekarnik', 'equipment_pralka', 'equipment_telewizor', 'equipment_zmywarka', 
    'media_types_brak informacji', 'media_types_internet', 'media_types_telefon', 'media_types_telewizja kablowa', 
    'heating_brak informacji', 'heating_elektryczne', 'heating_gazowe', 'heating_inne', 'heating_kotłownia', 'heating_miejskie', 'heating_piece kaflowe', 
    'security_brak informacji', 'security_domofon / wideofon', 'security_drzwi / okna antywłamaniowe', 'security_monitoring / ochrona', 'security_rolety antywłamaniowe', 'security_system alarmowy', 'security_teren zamknięty', 
    'windows_aluminiowe', 'windows_brak informacji', 'windows_drewniane', 'windows_plastikowe', 
    'building_material_beton', 'building_material_beton komórkowy', 'building_material_brak informacji', 'building_material_cegła', 'building_material_drewno', 'building_material_inne', 'building_material_keramzyt', 'building_material_pustak', 'building_material_silikat', 'building_material_wielka płyta', 'building_material_żelbet', 
    'additional_information_brak informacji', 'additional_information_dwupoziomowe', 'additional_information_klimatyzacja', 'additional_information_oddzielna kuchnia', 'additional_information_piwnica', 'additional_information_pom. użytkowe', 'additional_information_tylko dla niepalących'
]

# Add missing columns from columns_order with NaN values
for column in columns_order:
    if column not in df_otodom_cleaned.columns:
        df_otodom_cleaned[column] = np.nan
        
df_otodom_cleaned = df_otodom_cleaned[columns_order]

In [63]:
columns_multiindex = [
    ('listing', 'link'),
    ('listing', 'title'),
    ('listing', 'summary_description'),
    ('listing', 'remote_service'),
    ('pricing', 'price'),
    ('pricing', 'rent'),
    ('pricing', 'total_rent'),
    ('pricing', 'deposit'),
    ('location', 'complete_address'),
    ('location', 'street'),
    ('location', 'city'),
    ('location', 'voivodeship'),
    ('size', 'square_meters'),
    ('size', 'number_of_rooms'),
    ('size', 'floor'),
    ('size', 'attic'),
    ('size', 'building_floors'),
    ('legal_and_availability', 'available_from'),
    ('legal_and_availability', 'completion'),
    ('legal_and_availability', 'ownership'),
    ('legal_and_availability', 'rent_to_students'),
    ('type_and_year', 'building_type'),
    ('type_and_year', 'build_year'),
    ('amenities', 'elevator'),
    ('amenities', 'parking_space'),
    ('equipment', 'no_information'),
    ('equipment', 'stove'),
    ('equipment', 'fridge'),
    ('equipment', 'furniture'),
    ('equipment', 'oven'),
    ('equipment', 'washing_machine'),
    ('equipment', 'TV'),
    ('equipment', 'dishwasher'),
    ('media_types', 'no_information'),
    ('media_types', 'internet'),
    ('media_types', 'telephone'),
    ('media_types', 'cable_TV'),
    ('heating', 'no_information'),
    ('heating', 'electric'),
    ('heating', 'gas'),
    ('heating', 'other'),
    ('heating', 'boiler_room'),
    ('heating', 'district'),
    ('heating', 'tile_stove'),
    ('security', 'no_information'),
    ('security', 'intercom_or_video_intercom'),
    ('security', 'anti_burglary_doors_or_windows'),
    ('security', 'monitoring_or_security'),
    ('security', 'anti_burglary_roller_blinds'),
    ('security', 'alarm_system'),
    ('security', 'enclosed_area'),
    ('windows', 'aluminum'),
    ('windows', 'no_information'),
    ('windows', 'wooden'),
    ('windows', 'plastic'),
    ('building_material', 'concrete'),
    ('building_material', 'aerated_concrete'),
    ('building_material', 'no_information'),
    ('building_material', 'brick'),
    ('building_material', 'wood'),
    ('building_material', 'other'),
    ('building_material', 'lightweight_aggregate'),
    ('building_material', 'hollow_brick'),
    ('building_material', 'silicate'),
    ('building_material', 'large_panel'),
    ('building_material', 'reinforced_concrete'),
    ('additional_information', 'no_information'),
    ('additional_information', 'duplex'),
    ('additional_information', 'air_conditioning'),
    ('additional_information', 'separate_kitchen'),
    ('additional_information', 'basement'),
    ('additional_information', 'utility_room'),
    ('additional_information', 'non_smokers_only')
]

multiindex = pd.MultiIndex.from_tuples(columns_multiindex, names=['Category', 'Subcategory'])
df_otodom_cleaned.columns = multiindex

In [64]:
df_otodom_cleaned.head()

Category                                               listing  \
Subcategory                                               link   
0            https://www.otodom.pl/pl/oferta/bezposrednio-w...   
1            https://www.otodom.pl/pl/oferta/ciche-52m2przy...   
2            https://www.otodom.pl/pl/oferta/rezerwacja-m2-...   
3            https://www.otodom.pl/pl/oferta/3-oddzielne-po...   

Category                                                        \
Subcategory                                              title   
0                               BEZPOŚREDNIO, wola, 2 pokojowe   
1                Ciche 52m2przy kanale żerańskim - Bez Prowizj   
2            REZERWACJA - M2 44m2 | Balkon | Garaż | Praga ...   
3               3 oddzielne pok. - Park Picassa - Bez prowizji   

Category                                                        \
Subcategory                                summary_description   
0            Wynajmę bezpośrednio mieszkanie w Warszawie pr...   
1            Zamieszkaj w ciszy i dobrym standardzie\n\n   ...   
2            REZERWACJA!!!\n\nOferuję do wynajęcia dwupokoj...   
3            Zamieszkaj w ciszy w dobrym standardzie na pon...   

Category                           pricing                          \
Subcategory         remote_service   price rent total_rent deposit   
0            Obsługa zdalnaZapytaj  3000.0  NaN     3000.0     NaN   
1            Obsługa zdalnaZapytaj  2340.0  NaN     2340.0  2940.0   
2                Obsługa zdalnatak  3200.0  1.0     3201.0  3200.0   
3            Obsługa zdalnaZapytaj  3000.0  NaN     3000.0  3550.0   

Category                                              location  \
Subcategory                                   complete_address   
0            ul. Władysława Przanowskiego 83, Ulrychów, Wol...   
1            ul. Żeglugi Wiślanej, Kobiałka, Białołęka, War...   
2            ul. Ostrobramska, Gocław, Praga-Południe, Wars...   
3            ul. Erazma z Zakroczymia, Tarchomin, Białołęka...   

Category                                      ... building_material  \
Subcategory                           street  ...          silicate   
0            ul. Władysława Przanowskiego 83  ...               NaN   
1                       ul. Żeglugi Wiślanej  ...               NaN   
2                           ul. Ostrobramska  ...               NaN   
3                   ul. Erazma z Zakroczymia  ...               NaN   

Category                                    additional_information         \
Subcategory large_panel reinforced_concrete         no_information duplex   
0                   NaN                   0                      1    NaN   
1                   NaN                   0                      0    NaN   
2                   NaN                   1                      0    NaN   
3                   NaN                   0                      0    NaN   

Category                                                             \
Subcategory air_conditioning separate_kitchen basement utility_room   
0                        NaN                0      NaN          NaN   
1                        NaN                1      NaN          NaN   
2                        NaN                1      NaN          NaN   
3                        NaN                1      NaN          NaN   

Category                      
Subcategory non_smokers_only  
0                          0  
1                          1  
2                          0  
3                          1  

[4 rows x 73 columns]

In [65]:
df_otodom_cleaned.dtypes.to_dict()

{('listing', 'link'): dtype('O'),
 ('listing', 'title'): dtype('O'),
 ('listing', 'summary_description'): dtype('O'),
 ('listing', 'remote_service'): dtype('O'),
 ('pricing', 'price'): dtype('float64'),
 ('pricing', 'rent'): dtype('float64'),
 ('pricing', 'total_rent'): dtype('float64'),
 ('pricing', 'deposit'): dtype('float64'),
 ('location', 'complete_address'): dtype('O'),
 ('location', 'street'): dtype('O'),
 ('location', 'city'): dtype('O'),
 ('location', 'voivodeship'): dtype('O'),
 ('size', 'square_meters'): dtype('float64'),
 ('size', 'number_of_rooms'): Int64Dtype(),
 ('size', 'floor'): Int64Dtype(),
 ('size', 'attic'): dtype('bool'),
 ('size', 'building_floors'): Int64Dtype(),
 ('legal_and_availability', 'available_from'): dtype('O'),
 ('legal_and_availability', 'completion'): dtype('O'),
 ('legal_and_availability', 'ownership'): dtype('O'),
 ('legal_and_availability', 'rent_to_students'): dtype('O'),
 ('type_and_year', 'building_type'): dtype('O'),
 ('type_and_year', 'build_

#### 2.2.2 Checking data

##### Prices

In [66]:

assert df_otodom_cleaned[[('pricing', 'price'), ('pricing', 'rent'), ('pricing', 'deposit')]].min().min() >= 0, "Price, rent, or deposit contains negative values"

In [67]:
df_otodom_cleaned[[('pricing', 'price'), ('pricing', 'rent'), ('pricing', 'deposit')]].max()

Category  Subcategory
pricing   price          3200.0
          rent              1.0
          deposit        3550.0
dtype: float64

In [68]:
def last_and_first_percentile(column_name, df):
    """
    Returns the first and last percentile of a column in a DataFrame.

    Parameters:
    column_name (str): The name of the column to analyze.
    df (pandas.DataFrame): The DataFrame containing the column.

    Returns:
    tuple: A tuple containing the first and last percentile of the column.
    """
    return df[column_name].quantile([0.01, 0.99])

In [69]:
last_and_first_percentile(('pricing', 'price'), df_otodom_cleaned)

0.01    2359.8
0.99    3194.0
Name: (pricing, price), dtype: float64

Quick look

In [70]:
pd.set_option('display.max_colwidth', None)
df_otodom_cleaned.sort_values(by=[('pricing', 'price')], ascending=False).head()[[('listing', 'link'), ('listing', 'title'), ('listing', 'summary_description'), ('pricing', 'total_rent'), ('location', 'city')]]


Category                                                                                      listing  \
Subcategory                                                                                      link   
2              https://www.otodom.pl/pl/oferta/rezerwacja-m2-44m2-balkon-garaz-praga-pld-ID4oRGR.html   
0                           https://www.otodom.pl/pl/oferta/bezposrednio-wola-2-pokojowe-ID4oYsF.html   
3              https://www.otodom.pl/pl/oferta/3-oddzielne-pok-park-picassa-bez-prowizji-ID4oclG.html   
1            https://www.otodom.pl/pl/oferta/ciche-52m2przy-kanale-zeranskim-bez-prowizj-ID4oaVH.html   

Category                                                         \
Subcategory                                               title   
2            REZERWACJA - M2 44m2 | Balkon | Garaż | Praga Płd.   
0                                BEZPOŚREDNIO, wola, 2 pokojowe   
3                3 oddzielne pok. - Park Picassa - Bez prowizji   
1                 Ciche 52m2przy kanale żerańskim - Bez Prowizj   

Category                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        \
Subcategory                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [71]:
pd.set_option('display.max_colwidth', 50)

##### locations

In [72]:
set(df_otodom_cleaned[('location', 'city')])

{'Warszawa'}

In [73]:
set(df_otodom_cleaned[('location', 'voivodeship')])

{'mazowieckie'}

Textual Data Analysis

In [74]:
df_otodom_cleaned[('listing', 'summary_description')].str.len().max()

1715

In [75]:
df_otodom_cleaned[('listing', 'summary_description')].apply(count_words).max()

254

Max values of the selected columns

In [76]:
df_otodom_cleaned[('size', 'square_meters')].max()

63.0

In [77]:
df_otodom_cleaned[('size', 'square_meters')].min()

36.0

In [78]:
df_otodom_cleaned[('size', 'number_of_rooms')].max()

3

In [79]:
df_otodom_cleaned[('size', 'number_of_rooms')].min()

2

In [80]:
df_otodom_cleaned[('size', 'floor')].value_counts().index.to_list()

[2, 3, 4, 1]

In [81]:
df_otodom_cleaned[('size', 'building_floors')].value_counts()

3     2
18    1
10    1
Name: (size, building_floors), dtype: Int64

Check if date column is the date format

In [82]:
date_format_regex = r'^\d{4}-\d{2}-\d{2}$'

# Check if each date in the column matches the format
# Perform the assertion directly
assert (df_otodom_cleaned[('legal_and_availability', 'available_from')].dropna().str.match(date_format_regex)).all(), "Not all dates match the required format"


#####  2.2.3 Translate Polish to English
`Listing | title`, `Listing | summary_description` are not translated due to losing context by using a translation

listing

In [83]:
df_otodom_cleaned[('listing', 'remote_service')] = df_otodom_cleaned[('listing', 'remote_service')].map(
    {'Obsługa zdalnaZapytaj': np.NaN, 
     'Obsługa zdalnatak': 'unspecified', 
     'Obsługa zdalnaFilm': 'video',
     'Obsługa zdalnaWirtualny spacer': 'virtual_tour',
     'Obsługa zdalnaFilmWirtualny spacer': 'video_virtual_tour',
     }
    )
df_otodom_cleaned[('listing', 'remote_service')].value_counts(dropna=False)

NaN            3
unspecified    1
Name: (listing, remote_service), dtype: int64

legal_and_availability

In [84]:
df_otodom_cleaned[('legal_and_availability', 'completion')] = df_otodom_cleaned[('legal_and_availability', 'completion')].map(
    {'do zamieszkania': 'ready_to_move_in', 
     'do remontu': 'in_need_of_renovation', 
     'do wykończenia': 'unfinished'}
    )
df_otodom_cleaned[('legal_and_availability', 'completion')].value_counts()

ready_to_move_in    4
Name: (legal_and_availability, completion), dtype: int64

In [85]:
df_otodom_cleaned[('legal_and_availability', 'ownership')]= df_otodom_cleaned[('legal_and_availability', 'ownership')].map(
    {'biuro nieruchomości': 'real_estate_agency', 
     'prywatny': 'private', 
     'deweloper': 'developer'}
     )
df_otodom_cleaned[('legal_and_availability', 'ownership')].value_counts()

real_estate_agency    3
private               1
Name: (legal_and_availability, ownership), dtype: int64

In [86]:
df_otodom_cleaned[('legal_and_availability', 'rent_to_students')] = df_otodom_cleaned[('legal_and_availability', 'rent_to_students')].map({'brak informacji': np.NaN, 'tak': True, 'nie': False})
df_otodom_cleaned[('legal_and_availability', 'rent_to_students')].value_counts(dropna=False)

NaN    4
Name: (legal_and_availability, rent_to_students), dtype: int64

type_and_year

In [87]:
df_otodom_cleaned['type_and_year'].head()

Subcategory,building_type,build_year
0,blok,<NA>
1,blok,2011
2,blok,2004
3,blok,1993


In [88]:
df_otodom_cleaned[('type_and_year', 'building_type')].value_counts()

blok    4
Name: (type_and_year, building_type), dtype: int64

In [89]:
df_otodom_cleaned[('type_and_year', 'building_type')] = df_otodom_cleaned[('type_and_year', 'building_type')].map({
    'blok': 'block_of_flats', 
    'apartamentowiec': 'apartment_building', 
    'kamienica': 'historic_apartment_building',
    'dom wolnostojący': 'detached_house',
    'szeregowiec': 'terraced_house',
    })
df_otodom_cleaned[('type_and_year', 'building_type')].value_counts(dropna=False)

block_of_flats    4
Name: (type_and_year, building_type), dtype: int64

##### Change data types

bool

In [90]:
df_otodom_cleaned[('legal_and_availability', 'rent_to_students')] = df_otodom_cleaned[('legal_and_availability', 'rent_to_students')].fillna(0).astype('boolean')
df_otodom_cleaned[('legal_and_availability', 'rent_to_students')].head()

0    False
1    False
2    False
3    False
Name: (legal_and_availability, rent_to_students), dtype: boolean

In [91]:
df_otodom_cleaned[('legal_and_availability', 'rent_to_students')].value_counts(dropna=False)

False    4
<NA>     0
Name: (legal_and_availability, rent_to_students), dtype: Int64

In [92]:
df_otodom_cleaned['equipment'].head()

Subcategory,no_information,stove,fridge,furniture,oven,washing_machine,TV,dishwasher
0,NaN,1,1,1,1,1,NaN,0
1,NaN,1,1,1,1,1,NaN,1
2,NaN,1,1,1,1,1,NaN,0
3,NaN,1,1,1,1,1,NaN,1


In [93]:
for col in df_otodom_cleaned['equipment'].columns:
    df_otodom_cleaned[('equipment', col)] = df_otodom_cleaned[('equipment', col)].fillna(0).astype(bool)
df_otodom_cleaned['equipment'].head()

Subcategory,no_information,stove,fridge,furniture,oven,washing_machine,TV,dishwasher
0,False,True,True,True,True,True,False,False
1,False,True,True,True,True,True,False,True
2,False,True,True,True,True,True,False,False
3,False,True,True,True,True,True,False,True


In [94]:
for col in df_otodom_cleaned['media_types'].columns:
    df_otodom_cleaned[('media_types', col)] = df_otodom_cleaned[('media_types', col)].fillna(0).astype(bool)
df_otodom_cleaned['media_types'].head()

Subcategory,no_information,internet,telephone,cable_TV
0,False,True,False,True
1,True,False,False,False
2,False,True,False,False
3,True,False,False,False


In [95]:
for col in df_otodom_cleaned['heating'].columns:
    df_otodom_cleaned[('heating', col)] = df_otodom_cleaned[('heating', col)].fillna(0).astype(bool)
df_otodom_cleaned['heating'].head()

Subcategory,no_information,electric,gas,other,boiler_room,district,tile_stove
0,False,False,False,False,False,True,False
1,False,False,False,False,False,True,False
2,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False


In [96]:
for col in df_otodom_cleaned['security'].columns:
    df_otodom_cleaned[('security', col)] = df_otodom_cleaned[('security', col)].fillna(0).astype(bool)
df_otodom_cleaned['security'].head()

Subcategory,no_information,intercom_or_video_intercom,anti_burglary_doors_or_windows,monitoring_or_security,anti_burglary_roller_blinds,alarm_system,enclosed_area
0,True,False,False,False,False,False,False
1,False,True,False,True,False,False,True
2,False,True,False,True,False,False,True
3,False,True,False,False,False,False,False


In [97]:
for col in df_otodom_cleaned['windows'].columns:
    df_otodom_cleaned[('windows', col)] = df_otodom_cleaned[('windows', col)].fillna(0).astype(bool)
df_otodom_cleaned['windows'].head()

Subcategory,aluminum,no_information,wooden,plastic
0,False,False,True,False
1,False,False,False,True
2,False,False,False,True
3,False,False,False,True


In [98]:
for col in df_otodom_cleaned['building_material'].columns:
    df_otodom_cleaned[('building_material', col)] = df_otodom_cleaned[('building_material', col)].fillna(0).astype(bool)
df_otodom_cleaned['building_material'].head()

Subcategory,concrete,aerated_concrete,no_information,brick,wood,other,lightweight_aggregate,hollow_brick,silicate,large_panel,reinforced_concrete
0,False,False,False,True,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,True,False,False,False,False,False,False,False


In [99]:
for col in df_otodom_cleaned['additional_information'].columns:
    df_otodom_cleaned[('additional_information', col)] = df_otodom_cleaned[('additional_information', col)].fillna(0).astype(bool)
df_otodom_cleaned['additional_information'].head()

Subcategory,no_information,duplex,air_conditioning,separate_kitchen,basement,utility_room,non_smokers_only
0,True,False,False,False,False,False,False
1,False,False,False,True,False,False,True
2,False,False,False,True,False,False,False
3,False,False,False,True,False,False,True


Converting selected columns to the strings<br>
*We do not care about backward compatibly, and a `string` is much more readable than a `object`*

In [100]:
columns_to_convert = [
    ('listing', 'link'),
    ('listing', 'title'),
    ('listing', 'summary_description'),
    ('listing', 'remote_service'),
    ('location', 'complete_address'),
    ('location', 'street'),
    ('location', 'city'),
    ('location', 'voivodeship'),
    ('legal_and_availability', 'available_from'),
    ('legal_and_availability', 'completion'),
    ('legal_and_availability', 'ownership'),
    ('type_and_year', 'building_type'),
]

# Convert each column to the pandas string type
for col in columns_to_convert:
    df_otodom_cleaned[col] = df_otodom_cleaned[col].astype('string')

In [101]:
df_otodom_cleaned.dtypes.to_dict()

{('listing', 'link'): string[python],
 ('listing', 'title'): string[python],
 ('listing', 'summary_description'): string[python],
 ('listing', 'remote_service'): string[python],
 ('pricing', 'price'): dtype('float64'),
 ('pricing', 'rent'): dtype('float64'),
 ('pricing', 'total_rent'): dtype('float64'),
 ('pricing', 'deposit'): dtype('float64'),
 ('location', 'complete_address'): string[python],
 ('location', 'street'): string[python],
 ('location', 'city'): string[python],
 ('location', 'voivodeship'): string[python],
 ('size', 'square_meters'): dtype('float64'),
 ('size', 'number_of_rooms'): Int64Dtype(),
 ('size', 'floor'): Int64Dtype(),
 ('size', 'attic'): dtype('bool'),
 ('size', 'building_floors'): Int64Dtype(),
 ('legal_and_availability', 'available_from'): string[python],
 ('legal_and_availability', 'completion'): string[python],
 ('legal_and_availability', 'ownership'): string[python],
 ('legal_and_availability', 'rent_to_students'): BooleanDtype,
 ('type_and_year', 'building_

## 3. Save cleaned data

### 3.1. Save data

In [102]:
data_path_manager.save_df(df_otodom_cleaned, domain="otodom")

Saving schema to d:\UserData karol\Documents\Programming\Data Science\Data Engineering\Rent comparisions\Home Market Harvester\data\cleaned\2024_02_09_11_45_45_Warszawa\otodom_pl_schema.json
Saving CSV to d:\UserData karol\Documents\Programming\Data Science\Data Engineering\Rent comparisions\Home Market Harvester\data\cleaned\2024_02_09_11_45_45_Warszawa\otodom.pl.csv


### 3.2 Check saved data

### Otodom

In [103]:
df_otodom_saved = data_path_manager.load_df(domain="otodom", is_cleaned=True)
df_otodom_saved.head()


listing  \
                                                link   
0  https://www.otodom.pl/pl/oferta/bezposrednio-w...   
1  https://www.otodom.pl/pl/oferta/ciche-52m2przy...   
2  https://www.otodom.pl/pl/oferta/rezerwacja-m2-...   
3  https://www.otodom.pl/pl/oferta/3-oddzielne-po...   

                                                      \
                                               title   
0                     BEZPOŚREDNIO, wola, 2 pokojowe   
1      Ciche 52m2przy kanale żerańskim - Bez Prowizj   
2  REZERWACJA - M2 44m2 | Balkon | Garaż | Praga ...   
3     3 oddzielne pok. - Park Picassa - Bez prowizji   

                                                                    pricing  \
                                 summary_description remote_service   price   
0  Wynajmę bezpośrednio mieszkanie w Warszawie pr...           <NA>  3000.0   
1  Zamieszkaj w ciszy i dobrym standardzie

     ...           <NA>  2340.0   
2  REZERWACJA!!!

Oferuję do wynajęcia dwupokojow...    unspecified  3200.0   
3  Zamieszkaj w ciszy w dobrym standardzie na pon...           <NA>  3000.0   

                                                                    location  \
  rent total_rent deposit                                   complete_address   
0  NaN     3000.0     NaN  ul. Władysława Przanowskiego 83, Ulrychów, Wol...   
1  NaN     2340.0  2940.0  ul. Żeglugi Wiślanej, Kobiałka, Białołęka, War...   
2  1.0     3201.0  3200.0  ul. Ostrobramska, Gocław, Praga-Południe, Wars...   
3  NaN     3000.0  3550.0  ul. Erazma z Zakroczymia, Tarchomin, Białołęka...   

                                    ... building_material              \
                            street  ...          silicate large_panel   
0  ul. Władysława Przanowskiego 83  ...             False       False   
1             ul. Żeglugi Wiślanej  ...             False       False   
2                 ul. Ostrobramska  ...             False       False   
3         ul. Erazma z Zakroczymia  ...             False       False   

                      additional_information                          \
  reinforced_concrete         no_information duplex air_conditioning   
0               False                   True  False            False   
1               False                  False  False            False   
2                True                  False  False            False   
3               False                  False  False            False   

                                                           
  separate_kitchen basement utility_room non_smokers_only  
0            False    False        False            False  
1             True    False        False             True  
2             True    False        False            False  
3             True    False        False             True  

[4 rows x 73 columns]

In [104]:
are_identical = df_otodom_saved.equals(df_otodom_cleaned)
if not are_identical:
    raise ValueError("The saved DataFrame is not identical to the original one.")
else:
    print("The saved DataFrame is identical to the original one.")

The saved DataFrame is identical to the original one.
